In [ ]:
import pandas as pd
# Load the CSV file into a DataFrame
csv_file_path = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/Embedding/Processed_Dataset.csv'
df = pd.read_csv(csv_file_path)
# Display the first few rows of the DataFrame
df.head()

,Functional Description,Method Name,Predicted_Method,Prefix,Infix,Suffix,Prefix Probability,Infix Probability,Suffix Probability,Pivot,Mname,Edit_Score,Number
0,get the role of this object,get accessible role,getRole,get,NaN,NaN,0.87553,0.000000,0.0,"{'get': ('get', 0.6809051036834717), 'accessib...",getAccessibleRole,10,1
1,get the state of this object,get accessible state set,getState,get,NaN,NaN,0.87553,0.000000,0.0,"{'get': ('get', 0.6631520986557007), 'accessib...",getAccessibleStateSet,13,2
2,called by the browser or applet viewer to info...,destroy,informAppletOfReclaim,destroy,NaN,NaN,0.87500,0.000000,0.0,"{'destroy': ('allocated', 0.47457969188690186)}",destroy,20,3
3,gets the accessible context associated with th...,get accessible context,getAccessibleContext,get,accessible,NaN,0.87553,0.833333,0.0,"{'get': ('gets', 0.610834002494812), 'accessib...",getAccessibleContext,0,4
4,"determines this applet ' s context , which all...",get applet context,getAppletContext,NaN,NaN,NaN,0.00000,0.000000,0.0,"{'get': ('query', 0.4418052136898041), 'apple'...",getAppletContext,0,5


In [ ]:
pip install transformers torch scikit-learn

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import pickle
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
def get_embedding(text):
    """Generate an embedding for the given text using BERT."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the last hidden states as the embedding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

def extract_number_from_filename(filename):
    """Extract the number from a filename like 'embedding_4810.pkl'."""
    return int(filename.split('_')[1].split('.')[0])

def load_embeddings(embeddings_dir):
    """Load embeddings from pickle files and extract the Number index."""
    embeddings = []
    indices = []

    for file_name in os.listdir(embeddings_dir):
        if file_name.endswith('.pkl'):
            with open(os.path.join(embeddings_dir, file_name), 'rb') as file:
                embedding = pickle.load(file)
                embeddings.append(embedding)
                # Extract the Number index from the file name
                number_index = extract_number_from_filename(file_name)
                indices.append(number_index)

    return np.vstack(embeddings), indices

# Directory where the embeddings are saved
embeddings_dir = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/Embedding/'

# Load the saved embeddings and their Number indices from the files
dataset_embeddings, indices = load_embeddings(embeddings_dir)

def getnumber(text):
  input_embedding = get_embedding(text)
  # Compute cosine similarity scores
  similarity_scores = cosine_similarity(input_embedding, dataset_embeddings)[0]
  # Get the indices of the top 20 similar descriptions
  top_20_indices = np.argsort(similarity_scores)[-10:][::-1]
  # Select the top 20 Number indices
  top_20_numbers = [indices[i] for i in top_20_indices]
  return top_20_numbers


In [ ]:
# Load the original DataFrame
csv_path = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/Embedding/Processed_Dataset.csv'
df = pd.read_csv(csv_path)
df.head()

,Functional Description,Method Name,Predicted_Method,Prefix,Infix,Suffix,Prefix Probability,Infix Probability,Suffix Probability,Pivot,Mname,Edit_Score,Number
0,get the role of this object,get accessible role,getRole,get,NaN,NaN,0.87553,0.000000,0.0,"{'get': ('get', 0.6809051036834717), 'accessib...",getAccessibleRole,10,1
1,get the state of this object,get accessible state set,getState,get,NaN,NaN,0.87553,0.000000,0.0,"{'get': ('get', 0.6631520986557007), 'accessib...",getAccessibleStateSet,13,2
2,called by the browser or applet viewer to info...,destroy,informAppletOfReclaim,destroy,NaN,NaN,0.87500,0.000000,0.0,"{'destroy': ('allocated', 0.47457969188690186)}",destroy,20,3
3,gets the accessible context associated with th...,get accessible context,getAccessibleContext,get,accessible,NaN,0.87553,0.833333,0.0,"{'get': ('gets', 0.610834002494812), 'accessib...",getAccessibleContext,0,4
4,"determines this applet ' s context , which all...",get applet context,getAppletContext,NaN,NaN,NaN,0.00000,0.000000,0.0,"{'get': ('query', 0.4418052136898041), 'apple'...",getAppletContext,0,5


In [ ]:
def generate_prompt(row):
    # Ensure the input is treated as a DataFrame row
    if isinstance(row, pd.Series):
        row = row.to_frame().T

    s = f"Functional Description: {row['Functional Description'].values[0]}, Method Name: {row['Mname'].values[0]}"
    s+=" \n In this example "

    # Check and append prefix information
    if pd.notnull(row["Prefix"].values[0]) and row["Prefix"].values[0].strip():
        s += f" Prefix ('{row['Prefix'].values[0]}', Probability:{row['Prefix Probability'].values[0]}%)"

    # Check and append infix information
    if pd.notnull(row["Infix"].values[0]) and row["Infix"].values[0].strip():
        s += f", Infix ('{row['Infix'].values[0]}', Probability:{row['Infix Probability'].values[0]}%)"

    # Check and append suffix information
    if pd.notnull(row["Suffix"].values[0]) and row["Suffix"].values[0].strip():
        s += f" and Suffix ('{row['Suffix'].values[0]}', Probability:{row['Suffix Probability'].values[0]}%)"

    # Ensuring proper sentence ending
    if not s.endswith("."):
        s += "."

    # Append remaining details
    s += f" Semantic similarity between tokens of description and method name {row['Pivot'].values[0]}, "
    s += f" LLM predicted method name: {row['Predicted_Method'].values[0]} which has an edit distance score: {row['Edit_Score'].values[0]} as compared actual: {row['Mname'].values[0]}.\n"

    return s

# Example usage
def getlist(desls):
  s=""
  for i in desls:
    row = df.loc[df['Number'] == i]
    if isinstance(row, pd.Series):
        s=s+generate_prompt(row)
    else:
        for _, r in row.iterrows():
            s=s+generate_prompt(r)
  return s



In [ ]:
def promptcontext(text):
  s="Suggest a Java method name for this functional description: "+text +" \n To understand the method naming for this functional description, here are contextual relevant examples: \n"
  desls=getnumber(text)
  s=s+ getlist(desls)
  return s

In [ ]:
# Load the original DataFrame
csv_path = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/java_test.csv'
dftest = pd.read_csv(csv_path)
# Apply the promptcontext function to the first 300 rows of the 'Functional Description' column
dftest.loc[:299, 'Prompt'] = dftest.loc[:299, 'Functional Description'].apply(promptcontext)

# Save the modified DataFrame to a new CSV file
output_csv_path = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/java_test_prompt.csv'
dftest.to_csv(output_csv_path, index=False)

# Display a message indicating the process is complete
print(f"Processed DataFrame saved to {output_csv_path}")

Processed DataFrame saved to /content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/java_test_prompt.csv


In [ ]:
print(dfnumber)

[155, 154, 153, 920, 4059, 2959, 2908, 216, 4810, 2961, 2911, 4099, 2379, 917, 135, 254, 919, 541, 1115, 2963]


In [ ]:
top_20_numbers = [indices[i] for i in top_20_indices]
top_20_scores = [similarity_scores[i] for i in top_20_indices]

# Load the original DataFrame
csv_path = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/Embedding/Processed_Dataset.csv'
df = pd.read_csv(csv_path)

# Find and print the corresponding rows in the DataFrame
for i, (number, score) in enumerate(zip(top_20_numbers, top_20_scores), 1):
    row = df[df['Number'] == number]
    if not row.empty:
        print('Functional Description: "{row["Functional Description"]}"')
        print('Method Name: "{row["Mname"]}". ')
        print(f'In this example, "{row["Prefix"]}" is most likely to be the prefix with a probability of {row["Prefix Probability"]}%, ')
        print(f'"{row["Infix"]}" is the Infix with a probability of {row["Infix Probability"]}%, and ')
        print(f'"{row["Suffix"]}" is the Suffix with a probability of {row["Suffix Probability"]}% in the method name. ')
        print(f'More on, semantic similarity between description and method token is "{row["Pivot"]}". ')
        print(f'The base LLM predicted the method name "{row["Predicted_Method"]}", which has an edit distance score of {row["Edit_Score"]} ')
        print(f'compared to the ground truth "{row["Mname"]}".')

Functional Description: "{row["Functional Description"]}"
Method Name: "{row["Mname"]}". 
In this example, "2411    get
Name: Prefix, dtype: object" is most likely to be the prefix with a probability of 2411    0.87553
Name: Prefix Probability, dtype: float64%, 
"2411    NaN
Name: Infix, dtype: object" is the Infix with a probability of 2411    0.0
Name: Infix Probability, dtype: float64%, and 
"2411    NaN
Name: Suffix, dtype: object" is the Suffix with a probability of 2411    0.0
Name: Suffix Probability, dtype: float64% in the method name. 
More on, semantic similarity between description and method token is "2411    {'get': ('gets', 0.6878527402877808), 'renderi...
Name: Pivot, dtype: object". 
The base LLM predicted the method name "2411    getRenderingHints
Name: Predicted_Method, dtype: object", which has an edit distance score of 2411    0
Name: Edit_Score, dtype: int64 
compared to the ground truth "2411    getRenderingHints
Name: Mname, dtype: object".
Functional Description